In [22]:
import os
import json
import numpy as np

In [25]:
def f1_from_nonzero(A_true, A_est):
    A_true_nz = (A_true != 0)
    A_est_nz  = (A_est  != 0)

    TP = np.sum(A_true_nz & A_est_nz)
    FP = np.sum(~A_true_nz & A_est_nz)
    FN = np.sum(A_true_nz & ~A_est_nz)

    precision = TP / (TP + FP + 1e-12)
    recall    = TP / (TP + FN + 1e-12)
    f1        = 2 * precision * recall / (precision + recall + 1e-12)

    return TP, FP, FN, precision, recall, f1

In [27]:
folder = "result_d8_s7"  

W1_thres = 0.3
W2_thres = 0.2

# Track best files
best_f1_W1 = -1.0
best_f1_W1_file = None

best_f1_W2 = -1.0
best_f1_W2_file = None

best_avg_f1 = -1.0
best_avg_f1_file = None

for filename in os.listdir(folder):

    file_path = os.path.join(folder, filename)
    print("Processing:", file_path)

    with open(file_path, "r") as f:
        data = json.load(f)

    # --- truth ---
    W_truth = np.array(data["W_truth"])
    Sigma_truth = np.array(data["Sigma_truth"])  # assumes this key exists

    W_truth[np.abs(W_truth) < W1_thres] = 0
    Sigma_truth[np.abs(Sigma_truth) < W2_thres] = 0

    # --- pick run_id == 0 ---
    W_est = None
    Sigma_est = None
    chosen_run = None

    for run in data["random_runs"]:
        if run["run_id"] == 0:
            chosen_run = run
            W_est = np.array(run["W_est_random"])
            Sigma_est = np.array(run["Sigma_est_random"])
            break

    if W_est is None or Sigma_est is None:
        print(f"  Skipping {filename}: run_id == 0 not found.")
        continue

    # --- threshold estimates ---
    W_est[np.abs(W_est) < W1_thres] = 0
    Sigma_est[np.abs(Sigma_est) < W2_thres] = 0

    # --- F1 for W ---
    TP_W1, FP_W1, FN_W1, precision_W1, recall_W1, f1_W1 = f1_from_nonzero(W_truth, W_est)

    # --- F1 for Sigma (strictly lower-triangular) ---
    TP_W2, FP_W2, FN_W2, precision_W2, recall_W2, f1_W2 = f1_from_nonzero(
        np.tril(Sigma_truth, k=-1),
        np.tril(Sigma_est,   k=-1),
    )

    # --- save metrics into JSON ---
    data["f1_W1"] = round(f1_W1, 4)
    data["f1_W2"] = round(f1_W2, 4)
    # print("TP_W1", TP_W1)
    data["TP_W1"] = TP_W1.item()
    data["FP_W1"] = FP_W1.item()
    data["FN_W1"] = FN_W1.item()

    data["TP_W2"] = TP_W2.item()
    data["FP_W2"] = FP_W2.item()
    data["FN_W2"] = FN_W2.item()

    with open(file_path, "w") as f:
        json.dump(data, f, indent=2)

    # --- update best trackers ---
    avg_f1 = 0.5 * (f1_W1 + f1_W2)

    if f1_W1 > best_f1_W1:
        best_f1_W1 = f1_W1
        best_f1_W1_file = filename

    if f1_W2 > best_f1_W2:
        best_f1_W2 = f1_W2
        best_f1_W2_file = filename

    if avg_f1 > best_avg_f1:
        best_avg_f1 = avg_f1
        best_avg_f1_file = filename

# After the loop, print results
print("\n=== Best files ===")
print(f"Best f1_W1: {best_f1_W1:.4f} in file {best_f1_W1_file}")
print(f"Best f1_W2: {best_f1_W2:.4f} in file {best_f1_W2_file}")
print(f"Best average (f1_W1 + f1_W2)/2: {best_avg_f1:.4f} in file {best_avg_f1_file}")

Processing: result_d8_s7/result_d8_admg7_seed7_lambda0.001_lambda_corr0.01_lambda_nl4.0.json
Processing: result_d8_s7/result_d8_admg7_seed7_lambda0.001_lambda_corr0.01_lambda_nl8.0.json
Processing: result_d8_s7/result_d8_admg7_seed7_lambda0.1_lambda_corr0.001_lambda_nl4.0.json
Processing: result_d8_s7/result_d8_admg7_seed7_lambda0.01_lambda_corr0.1_lambda_nl4.0.json
Processing: result_d8_s7/result_d8_admg7_seed7_lambda0.001_lambda_corr0.001_lambda_nl4.0.json
Processing: result_d8_s7/result_d8_admg7_seed7_lambda0.1_lambda_corr0.01_lambda_nl4.0.json
Processing: result_d8_s7/result_d8_admg7_seed7_lambda0.001_lambda_corr0.1_lambda_nl4.0.json
Processing: result_d8_s7/result_d8_admg7_seed7_lambda0.1_lambda_corr0.01_lambda_nl8.0.json
Processing: result_d8_s7/result_d8_admg7_seed7_lambda0.01_lambda_corr0.01_lambda_nl8.0.json
Processing: result_d8_s7/result_d8_admg7_seed7_lambda0.01_lambda_corr0.1_lambda_nl8.0.json
Processing: result_d8_s7/result_d8_admg7_seed7_lambda0.1_lambda_corr0.001_lambda

In [28]:
folder = "result_d8_s7"  
scores = []

for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    print("Processing:", file_path)
    with open(file_path, "r") as f:
        data = json.load(f)
        f1_W1 = data["f1_W1"]
        f1_W2 = data["f1_W2"]
        avg_f1 = 0.5 * (f1_W1 + f1_W2)

    scores.append((filename, f1_W1, f1_W2, avg_f1))

best_f1_W1 = max(s[1] for s in scores)
best_f1_W2 = max(s[2] for s in scores)
best_avg_f1 = max(s[3] for s in scores)

best_f1_W1_files = [f for f, s1, _, _ in scores if s1 == best_f1_W1]
best_f1_W2_files = [f for f, _, s2, _ in scores if s2 == best_f1_W2]
best_avg_f1_files = [f for f, _, _, a in scores if a == best_avg_f1]




Processing: result_d8_s7/result_d8_admg7_seed7_lambda0.001_lambda_corr0.01_lambda_nl4.0.json
Processing: result_d8_s7/result_d8_admg7_seed7_lambda0.001_lambda_corr0.01_lambda_nl8.0.json
Processing: result_d8_s7/result_d8_admg7_seed7_lambda0.1_lambda_corr0.001_lambda_nl4.0.json
Processing: result_d8_s7/result_d8_admg7_seed7_lambda0.01_lambda_corr0.1_lambda_nl4.0.json
Processing: result_d8_s7/result_d8_admg7_seed7_lambda0.001_lambda_corr0.001_lambda_nl4.0.json
Processing: result_d8_s7/result_d8_admg7_seed7_lambda0.1_lambda_corr0.01_lambda_nl4.0.json
Processing: result_d8_s7/result_d8_admg7_seed7_lambda0.001_lambda_corr0.1_lambda_nl4.0.json
Processing: result_d8_s7/result_d8_admg7_seed7_lambda0.1_lambda_corr0.01_lambda_nl8.0.json
Processing: result_d8_s7/result_d8_admg7_seed7_lambda0.01_lambda_corr0.01_lambda_nl8.0.json
Processing: result_d8_s7/result_d8_admg7_seed7_lambda0.01_lambda_corr0.1_lambda_nl8.0.json
Processing: result_d8_s7/result_d8_admg7_seed7_lambda0.1_lambda_corr0.001_lambda

In [30]:
print(f"Best f1_W1: {best_f1_W1:.4f} in files:\n" +
      "\n".join(best_f1_W1_files))

Best f1_W1: 1.0000 in files:
result_d8_admg7_seed7_lambda0.01_lambda_corr0.01_lambda_nl4.0.json


In [31]:
print(f"Best f1_W2: {best_f1_W2:.4f} in files:\n" +
      "\n".join(best_f1_W2_files))

Best f1_W2: 0.8000 in files:
result_d8_admg7_seed7_lambda0.001_lambda_corr0.01_lambda_nl8.0.json
result_d8_admg7_seed7_lambda0.01_lambda_corr0.01_lambda_nl8.0.json
result_d8_admg7_seed7_lambda0.01_lambda_corr0.1_lambda_nl8.0.json
result_d8_admg7_seed7_lambda0.001_lambda_corr0.1_lambda_nl8.0.json


In [32]:
print(f"Best avg_f1: {best_avg_f1:.4f} in files:\n" +
      "\n".join(best_avg_f1_files))

Best avg_f1: 0.7333 in files:
result_d8_admg7_seed7_lambda0.001_lambda_corr0.01_lambda_nl8.0.json
result_d8_admg7_seed7_lambda0.01_lambda_corr0.01_lambda_nl8.0.json
result_d8_admg7_seed7_lambda0.01_lambda_corr0.1_lambda_nl8.0.json
result_d8_admg7_seed7_lambda0.001_lambda_corr0.1_lambda_nl8.0.json


In [33]:
filename = f"result_d8_admg7_seed7_lambda0.001_lambda_corr0.1_lambda_nl8.0.json" 
file_path = os.path.join(folder, filename)
with open(file_path, "r") as f:
        data = json.load(f)

for run in data["random_runs"]:
        if run["run_id"] == 0:
                chosen_run = run
                W_est = np.array(run["W_est_random"])
                Sigma_est = np.array(run["Sigma_est_random"])
                break
f1_W1 = data["f1_W1"]
f1_W2 = data["f1_W2"]
print("f1_W1: ", f1_W1)
print("f1_W2: ", f1_W2)
print("W_est: ", W_est)
print("Sigma_est: ", Sigma_est)

f1_W1:  0.6667
f1_W2:  0.8
W_est:  [[6.58843963e-16 4.28385583e-07 9.77700730e-06 1.00923210e-06
  6.89451486e-09 1.78826692e-05 1.82529268e-10 2.04884508e-07]
 [2.57767191e-08 1.30052390e-11 3.78006263e-05 1.20087161e-05
  1.15806072e-09 1.39484028e+00 3.27301319e-11 1.75748386e-06]
 [1.02243088e-07 3.28364440e-05 1.89150599e-11 4.57222806e+00
  1.26975397e-09 7.17924168e-06 1.01307216e-09 9.96366628e-07]
 [3.69024042e-08 4.65138183e-05 8.55990824e-05 1.09983672e-10
  1.97875460e-09 1.09326562e-04 1.01886090e-09 5.74819332e-07]
 [1.23658042e-07 1.35544223e-05 7.11404073e-07 1.84752155e-05
  1.02170578e-17 2.75716226e-06 2.70703691e-09 2.27224713e-05]
 [1.46270653e-08 6.19126452e-04 3.75314146e-06 1.21142331e-06
  1.54261846e-09 1.13457999e-10 2.10329037e-09 8.06065303e-07]
 [6.78827729e-08 5.58885147e-06 5.09613707e-05 9.53973320e-06
  1.32212347e-09 1.01784685e-05 1.18969439e-17 2.91642792e-06]
 [2.05477781e-08 2.25784299e-05 1.10832176e-04 2.63837746e-05
  4.06608550e-09 2.37927791e

## Overall f1 

In [ ]:
folder = "result_d4"  

W1_thres = 0.3
W2_thres = 0.2

# Track best files
best_f1_W1 = -1.0
best_f1_W1_file = None

best_f1_W2 = -1.0
best_f1_W2_file = None

best_avg_f1 = -1.0
best_avg_f1_file = None

for filename in os.listdir(folder):

    file_path = os.path.join(folder, filename)
    print("Processing:", file_path)

    with open(file_path, "r") as f:
        data = json.load(f)

    # --- truth ---
    W_truth = np.array(data["W_truth"])
    Sigma_truth = np.array(data["Sigma_truth"])  # assumes this key exists

    W_truth[np.abs(W_truth) < W1_thres] = 0
    Sigma_truth[np.abs(Sigma_truth) < W2_thres] = 0

    # --- pick run_id == 0 ---
    W_est = None
    Sigma_est = None
    chosen_run = None

    for run in data["random_runs"]:
        if run["run_id"] == 0:
            chosen_run = run
            W_est = np.array(run["W_est_random"])
            Sigma_est = np.array(run["Sigma_est_random"])
            break

    if W_est is None or Sigma_est is None:
        print(f"  Skipping {filename}: run_id == 0 not found.")
        continue

    # --- threshold estimates ---
    W_est[np.abs(W_est) < W1_thres] = 0
    Sigma_est[np.abs(Sigma_est) < W2_thres] = 0

    # --- F1 for W ---
    TP_W1, FP_W1, FN_W1, precision_W1, recall_W1, f1_W1 = f1_from_nonzero(W_truth, W_est)

    # --- F1 for Sigma (strictly lower-triangular) ---
    TP_W2, FP_W2, FN_W2, precision_W2, recall_W2, f1_W2 = f1_from_nonzero(
        np.tril(Sigma_truth, k=-1),
        np.tril(Sigma_est,   k=-1),
    )

    # --- save metrics into JSON ---
    data["f1_W1"] = round(f1_W1, 4)
    data["f1_W2"] = round(f1_W2, 4)
    # print("TP_W1", TP_W1)
    data["TP_W1"] = TP_W1.item()
    data["FP_W1"] = FP_W1.item()
    data["FN_W1"] = FN_W1.item()

    data["TP_W2"] = TP_W2.item()
    data["FP_W2"] = FP_W2.item()
    data["FN_W2"] = FN_W2.item()

    with open(file_path, "w") as f:
        json.dump(data, f, indent=2)

    # --- update best trackers ---
    avg_f1 = 0.5 * (f1_W1 + f1_W2)

    if f1_W1 > best_f1_W1:
        best_f1_W1 = f1_W1
        best_f1_W1_file = filename

    if f1_W2 > best_f1_W2:
        best_f1_W2 = f1_W2
        best_f1_W2_file = filename

    if avg_f1 > best_avg_f1:
        best_avg_f1 = avg_f1
        best_avg_f1_file = filename

# After the loop, print results
print("\n=== Best files ===")
print(f"Best f1_W1: {best_f1_W1:.4f} in file {best_f1_W1_file}")
print(f"Best f1_W2: {best_f1_W2:.4f} in file {best_f1_W2_file}")
print(f"Best average (f1_W1 + f1_W2)/2: {best_avg_f1:.4f} in file {best_avg_f1_file}")

In [21]:
TP_tot_W1 = 0
FP_tot_W1 = 0
FN_tot_W1 = 0

TP_tot_W2 = 0
FP_tot_W2 = 0
FN_tot_W2 = 0

for filename in os.listdir(folder):
    with open(os.path.join(folder, filename)) as f:
        data = json.load(f)

    TP_tot_W1 += data["TP_W1"]
    FP_tot_W1 += data["FP_W1"]
    FN_tot_W1 += data["FN_W1"]

    TP_tot_W2 += data["TP_W2"]
    FP_tot_W2 += data["FP_W2"]
    FN_tot_W2 += data["FN_W2"]

precision_W1 = TP_tot_W1 / (TP_tot_W1 + FP_tot_W1)
recall_W1    = TP_tot_W1 / (TP_tot_W1 + FN_tot_W1)
f1_W1        = 2 * precision_W1 * recall_W1 / (precision_W1+ recall_W1)

precision_W2 = TP_tot_W2 / (TP_tot_W2 + FP_tot_W2)
recall_W2    = TP_tot_W2 / (TP_tot_W2 + FN_tot_W2)
f1_W2        = 2 * precision_W2 * recall_W2 / (precision_W2+ recall_W2)

print("F1_W1:", f1_W1)
print("F1_W2:", f1_W2)


F1_W1: 0.8089887640449439
F1_W2: 0.8333333333333334
